# Web scraping
*Done by Océane Salmeron, December 2020*

In [23]:
#!pip install selenium
import pandas as pd
import numpy as np
import xlsxwriter
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [24]:
url = 'https://www.ycombinator.com/companies/'

#driver = webdriver.Chrome('./chromedriver')
caps = DesiredCapabilities().CHROME
# caps["pageLoadStrategy"] = "normal"  #  Waits for full page load
caps["pageLoadStrategy"] = "eager"   # Do not wait for full page load
driver = webdriver.Chrome(desired_capabilities=caps, executable_path='../chromedriver')
driver.get(url)

In [25]:
def scroll_to_end(driver):
    while len(driver.find_elements_by_xpath("//a[@class='SharedDirectory-module__company___AVmr6 no-hovercard']"))<500 :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [26]:
scroll_to_end(driver)

In [27]:
results = driver.find_elements_by_xpath("//a[@class='SharedDirectory-module__company___AVmr6 no-hovercard']")
i=0
links = []
for result in results:
    s=result.get_attribute('href')
    links.append(s.split('/')[-1])
    i+=1
print(i)

500


In [28]:
# Loop through all the links and launch one by one

dic = []

for link in links:
    
    driver.get(url+link)
    
    facts = driver.find_elements_by_css_selector(".facts div span")
    socials = driver.find_elements_by_css_selector(".social")
       
    item = {'name': driver.find_element_by_class_name("heavy").text, 
            'info': driver.find_element_by_css_selector(".main-box h3").text,
            'description': driver.find_element_by_class_name("pre-line").text,
            'website': driver.find_element_by_css_selector(".main-box .links a").get_attribute('href'),
            'launch_year': facts[0].text,
            'team_size': facts[1].text,
            'location': facts[2].text
           }
    
    for social in socials:
        item[social.get_attribute('class').split()[-1]] = social.get_attribute('href')
        
    dic.append(item)  

In [29]:
data = pd.DataFrame.from_dict(dic)
data.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [30]:
data.head()

,name,info,description,website,launch_year,team_size,location,linkedin,twitter,facebook,crunchbase
0,DoorDash,Restaurant delivery.,"Founded in 2013, DoorDash is a San Francisco-b...",http://doordash.com/,NaN,1600,San Francisco,https://www.linkedin.com/company/doordash/,http://twitter.com/doordash,https://www.facebook.com/DoorDash/,https://www.crunchbase.com/organization/doordash
1,Dropbox,Backup and share files in the cloud.,Dropbox is building the world’s first smart wo...,http://dropbox.com/,2008,4000,San Francisco,https://www.linkedin.com/in/drewhouston/,https://twitter.com/drewhouston,https://www.facebook.com/Dropbox/,https://www.crunchbase.com/organization/dropbox
2,Airbnb,Book accommodations around the world,Founded in August of 2008 and based in San Fra...,http://airbnb.com/,2008,5000,San Francisco,https://www.linkedin.com/in/blecharczyk/,https://twitter.com/jgebbia,https://www.facebook.com/airbnb/,https://www.crunchbase.com/organization/airbnb
3,PagerDuty,Notify you about server troubles.,PagerDuty is an operations performance platfor...,http://pagerduty.com/,NaN,775,San Francisco,https://www.linkedin.com/in/baskarfx/,NaN,NaN,https://www.crunchbase.com/organization/pagerduty
4,Embark Trucks,We build self-driving semi trucks.,We are a San Francisco based team building sel...,http://embarktrucks.com/,2016,100,San Francisco,https://ca.linkedin.com/in/rodriguesalex,NaN,NaN,https://www.crunchbase.com/organization/varden...


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         500 non-null    object
 1   info         495 non-null    object
 2   description  479 non-null    object
 3   website      499 non-null    object
 4   launch_year  291 non-null    object
 5   team_size    490 non-null    object
 6   location     492 non-null    object
 7   linkedin     400 non-null    object
 8   twitter      339 non-null    object
 9   facebook     191 non-null    object
 10  crunchbase   418 non-null    object
dtypes: object(11)
memory usage: 43.1+ KB


In [32]:
def export_excel(Filename,Sheetname,df,ind):
    try:
        xlsx_file = pd.ExcelFile(Filename)
    except:
        xlsxwriter.Workbook(Filename)
        xlsx_file = pd.ExcelFile(Filename)

    writer = pd.ExcelWriter(Filename, engine='openpyxl')
    IsSheetThereAlready = False
    for sheet in xlsx_file.sheet_names:
        if sheet == Sheetname:
            df.to_excel(writer,sheet_name= sheet, startrow=0, index=ind)
            IsSheetThereAlready = True

        else:
            df2 = xlsx_file.parse(sheet)
            df2.to_excel(writer,sheet_name= sheet, index=False)


    if IsSheetThereAlready is False:
        df.to_excel(writer,sheet_name = Sheetname, index=False)

    writer.save()

    return

In [33]:
filename = '../Data/Results.xlsx'

print('Starting writing to excel...')
export_excel(filename,'Scraping results', data, False)

Starting writing to excel...
